### **Setting Up Your Workspace**

In [8]:
import pandas as pd
import pandas_ta as ta
import pytz
from datetime import datetime, time, timedelta

### **Step 1: Load the Data and Make it Timezone-Aware**

Think of your raw data from MT5 as having a "naive" timestamp. It's just a number. We need to give it a "birth certificate" that says, "This time is in the UTC timezone." This is the most important first step.

In [9]:
# --- Step 1: Load Data and Set Timezone ---
print('Loading raw data...')
year = 2015
# Load the hourly data downloaded earlier using fastparquet(pip install this to allow pandas to use for the data loading)
df_raw = pd.read_parquet(f'../data/raw/xauusd_h1_{year}_present.parquet')

# The 'time' column is our master key. We make it the index of the Dataframe.
df_raw.set_index('time', inplace=True)


"""
Localize the naive index to UTC

This is to deal with forex timezones, standard timezone and Daylight Saving Time(DST) timezones affecting the London & New York time.

CRITICAL: Tell pandas that the existing timestamps are in UTC.
This doesn't change the numbers, it just adds the context.
"""
df = df_raw.tz_localize('UTC')

print(f"Data loaded successfully. Index timezone is now: {df.index.tz}")


Loading raw data...
Data loaded successfully. Index timezone is now: UTC


### **2. Pre-calculate technical indicators**

It's much faster to calculate indicators like RSI and ATR on the entire dataset at once, rather than re-calculating them inside our loop every day. We'll use the pandas_ta library for this.

In [10]:
# --- Step 2: Pre-calculate Technical Indicators ---
print("\nStep 2: Calculating technical indicators...")

# We use the .ta extension provided by the pandas_ta library.
# The `append=True` argument tells it to add the new columns directly to our DataFrame.
# `length` = lookback period (# of candles).

# Exponential Moving Average, EMA: trend strength clue (are we far from the mean?).
df.ta.ema(length=50, append=True)
df.ta.ema(length=200, append=True)

# Relative Single Index, RSI: momentum exhaustion clue (are we stretched?).
df.ta.rsi(length=14, append=True)

# Average Time Range, ATR: volatility momentum clue (is volatility expanding or contracting?)
df.ta.atr(length=14, append=True)

""" 
- length = lookback period (# of candles).
- EMA50/200 → short vs long trend.
- RSI14 → momentum/overbought/oversold.
- ATR14 → volatility.
- MACD + Bollinger aren’t strictly necessary, but worth experimenting with later for potential edge.

"""

print("Indicators calculated. New columns added.")
df.head() # Use .tail() to see the latest calculated values


Step 2: Calculating technical indicators...
Indicators calculated. New columns added.


,open,high,low,close,tick_volume,spread,real_volume,EMA_50,EMA_200,RSI_14,ATRr_14
time,,,,,,,,,,,
2015-01-02 09:00:00+00:00,1186.94,1187.15,1184.67,1185.85,3226,33,1376300,NaN,NaN,NaN,NaN
2015-01-02 10:00:00+00:00,1185.85,1188.00,1185.85,1186.60,3712,33,1538500,NaN,NaN,100.000000,NaN
2015-01-02 11:00:00+00:00,1186.60,1186.99,1184.75,1186.23,3514,33,1535800,NaN,NaN,96.343874,NaN
2015-01-02 12:00:00+00:00,1186.25,1186.78,1181.20,1182.57,4479,32,1903400,NaN,NaN,69.338074,NaN
2015-01-02 13:00:00+00:00,1182.55,1183.79,1180.57,1182.68,5121,33,2379600,NaN,NaN,69.613755,NaN


### **Step 3: The Main Loop - The Heart of the Script**

Now we'll go through our data day by day. For each day, we'll perform our session logic.

In [11]:
# --- Step 3: Loop Through Each Day to Engineer Features ---
print("\nStep 3: Starting daily feature engineering loop...")

# This list will hold the dictionary for each day's calculated data.
daily_data_list = []

# Define the local timezones we need.
london_tz = pytz.timezone('Europe/London')
# ny_tz = pytz.timezone("America/New_York") # for New York


# df.index.date gives us just the date part (e.g., 2025-09-20)
# We group by this to process one day at a time.
for day in df.index.normalize().unique(): # do 10 for now to visualise data
    # .normalize() sets the time to 00:00:00, ensuring one entry per day
    # .unique() gets each of those unique days.
    
    # We'll put all our logic for a single day inside this 'try...except' block.
    # This prevents the whole script from crashing if one day has bad data (e.g., a holiday).
    
    try:
        # --- 3a. Define the DYNAMIC London Session for this specific day ---
        # Create the start time: 8:00 AM in London on the current 'day'.
        london_open_local = london_tz.localize(datetime.combine(day, time(8, 0)))
        # Create the end time: 5:00 PM (17:00) in London on the current 'day'.
        london_close_local = london_tz.localize(datetime.combine(day, time(17, 0)))

        # Now, ask the question: "What time was that in UTC?"
        london_open_utc = london_open_local.astimezone(pytz.utc)
        london_close_utc = london_close_local.astimezone(pytz.utc)
        
        
        # --- 3b. Define the Asian Session (this is the trickiest part) ---
        # The Asian session starts the evening before.
        previous_day = day - timedelta(days=1)
        
        # print(f'day = {day}, timedelata(day=1) = {timedelta(days=1)}')
        # print(f'previous day = {previous_day}')
        
        # Get the evening hours from the previous day (e.g., 22:00 and 23:00) 
        asia_part1 = df.loc[str(previous_day.date())].between_time('22:00', '23:59') 
        # print(f'asia part 1 = {asia_part1}')
        
        # Get the morning hours from the current day (e.g., 00:00 to 07:59) 
        asia_part2 = df.loc[str(day.date())].between_time('00:00', '07:59') 
        # print(f'asia part 2 = {asia_part2}')
        
        # Stitch them together into one session.
        asia_session = pd.concat([asia_part1, asia_part2])
        # print(f'asia session = {asia_session.head(50)}')
        
        
        # --- 3c. Get the data slices (the "Guard Clause") ---
        london_session = df[(df.index >= london_open_utc) & (df.index < london_close_utc)]
        # print(f'london session = {london_session.head(50)}')

        # If either session is empty (e.g., weekend, holiday), skip this day.
        if asia_session.empty or london_session.empty:
            continue
        
        
        # # --- 3d. Calculate FEATURES from the Asian Session ---
        asia_open = asia_session['open'].iloc[0] # First price of the session
        asia_close = asia_session['close'].iloc[-1] # Last price
        asia_high = asia_session['high'].max()
        asia_low = asia_session['low'].min()

        asia_return = (asia_close - asia_open) / asia_open
        asia_range = asia_high - asia_low

        # Get the exact timestamp for the end of the Asian session to look up indicators
        end_of_asia_timestamp = asia_session.index[-1]
        # print("your end here ->", end_of_asia_timestamp)

        # Look up the pre-calculated indicator values at that specific time
        atr_at_asia_close = df.loc[end_of_asia_timestamp]['ATRr_14']
        rsi_at_asia_close = df.loc[end_of_asia_timestamp]['RSI_14']
        ema50_val = df.loc[end_of_asia_timestamp]['EMA_50']
        ema200_val = df.loc[end_of_asia_timestamp]['EMA_200']
        
        ema50_dist = (asia_close - ema50_val) / ema50_val # how are they calculating this though, curious
        ema200_dist = (asia_close - ema200_val) / ema200_val # and this guy
        
        
        # --- 3e. Calculate TARGETS from the London Session ---
        london_open = london_session['open'].iloc[0]
        london_close = london_session['close'].iloc[-1]
        
        # Target for Classification (Hypothesis A1)
        london_direction = 1 if london_close > london_open else 0 # close > open mean a bullish trend and visa versa
        
        # Target for Regression (Hypothesis A2)
        london_return = (london_close - london_open) / london_open # esitmate the return for the overall London session
        
        # --- 3f. Store everything in a dictionary ---
        daily_data_list.append({
            'date': day.date(),
            'day_of_week': day.dayofweek, # Monday=0, Sunday=6
            'asia_return': asia_return,
            'asia_range': asia_range,
            'atr_at_asia_close': atr_at_asia_close,
            'rsi_at_asia_close': rsi_at_asia_close,
            'ema50_dist': ema50_dist,
            'ema200_dist': ema200_dist,
            'london_direction': london_direction, # Our first target
            'london_return': london_return        # Our second target
        })
        
        
    except Exception as e:
        # If anything goes wrong for a specific day, print it and continue.
        print(f"Could not process {day.date()}: {e}")
        continue # Move to the next day
    
print(f"Loop finished. Processed {len(daily_data_list)} trading days.")


Step 3: Starting daily feature engineering loop...
Could not process 2015-01-02: '2015-01-01'
Loop finished. Processed 2762 trading days.


### **Step 4: Final Assembly and Saving**
The final step is to convert our list of dictionaries into a clean, final DataFrame and save it.

In [12]:
len(daily_data_list)

2762

In [13]:
# --- Step 4: Assemble Final DataFrame and Save ---
print("\nStep 4: Assembling and saving final feature DataFrame...")

# This is a very efficient way to create a DataFrame from our loop's results.
final_df = pd.DataFrame(daily_data_list)

# Before we set the index, we convert the 'date' column from generic 'object'
# to a proper 'datetime64' format that Parquet understands.
final_df['date'] = pd.to_datetime(final_df['date'])
final_df.set_index('date', inplace=True)

# The first few days of our data won't have EMA_200 values, because it needs 200 hours of data to "warm up".
# .dropna() removes these rows with missing values, ensuring our data is clean for the model.
final_df.dropna(inplace=True)

# year = 2015
# Save the final, clean dataset. We'll use this for training our models.
output_path = f'../data/processed/hyp_a_features_{year}_present.parquet'
final_df.to_parquet(output_path)

print(f"Successfully saved feature data to {output_path}")
print("--- Final DataFrame Info ---")
final_df.info()
print("\n--- Final DataFrame Head ---")
final_df


Step 4: Assembling and saving final feature DataFrame...
Successfully saved feature data to ../data/processed/hyp_a_features_2015_present.parquet
--- Final DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2754 entries, 2015-01-15 to 2025-09-19
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   day_of_week        2754 non-null   int64  
 1   asia_return        2754 non-null   float64
 2   asia_range         2754 non-null   float64
 3   atr_at_asia_close  2754 non-null   float64
 4   rsi_at_asia_close  2754 non-null   float64
 5   ema50_dist         2754 non-null   float64
 6   ema200_dist        2754 non-null   float64
 7   london_direction   2754 non-null   int64  
 8   london_return      2754 non-null   float64
dtypes: float64(7), int64(2)
memory usage: 215.2 KB

--- Final DataFrame Head ---


,day_of_week,asia_return,asia_range,atr_at_asia_close,rsi_at_asia_close,ema50_dist,ema200_dist,london_direction,london_return
date,,,,,,,,,
2015-01-15,3,-0.002831,6.29,3.920000,45.151821,-0.001292,0.011284,1,0.021539
2015-01-16,4,-0.000747,7.88,4.669597,59.367312,0.009069,0.027384,1,0.013524
2015-01-19,0,-0.001158,4.01,4.021460,64.454916,0.012683,0.035894,0,-0.001300
2015-01-21,2,0.004155,12.24,3.933239,80.992311,0.020631,0.047085,0,-0.004953
2015-01-22,3,-0.000735,6.03,3.739837,49.811537,0.004250,0.031106,1,0.002555
...,...,...,...,...,...,...,...,...,...
2025-09-15,0,0.000266,20.14,6.843659,53.301510,0.001228,0.014456,1,0.001141
2025-09-16,1,0.002015,14.72,7.797400,65.886681,0.007152,0.020984,1,0.002206
2025-09-17,2,-0.002680,20.93,7.032766,46.576228,0.001180,0.015342,0,-0.000845
